Copyright (c) 2023, DKRZ, MPI-M

All rights reserved.

Redistribution and use in source and binary forms, with or without modification,
are permitted provided that the following conditions are met:

1. Redistributions of source code must retain the above copyright notice, this
   list of conditions and the following disclaimer.

2. Redistributions in binary form must reproduce the above copyright notice, this
   list of conditions and the following disclaimer in the documentation and/or
   other materials provided with the distribution.

3. Neither the name of the copyright holder nor the names of its contributors
   may be used to endorse or promote products derived from this software without
   specific prior written permission.

THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND
ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED
WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED.
IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT,
INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING,
BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE,
DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY
OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE
OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF
ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.

In [ ]:
import xarray as xr
import isodate
from datetime import date
from os.path import exists

**ICON/YAC preamble**

In [ ]:
VERBOSE = 2

iso_start_date='1979-11-01T00:00:00.000'
iso_end_date='1980-04-01T00:00:00.000'
iso_coupling_interval='P1D'
iso_data_interval='P1M'

dataPath = "/pool/data/ICON/grids/public/mpim/independent/aerosol_kinne/"
fileRoot = "aeropt_kinne"

irad_aero = 12
lyr_perp = False
yr_perp = 1950

if ( irad_aero == 12 or irad_aero == 18 or irad_aero == 19 ) :
    scenario = "picontrol" # Kinne background from 1850

if ( irad_aero == 13 or irad_aero == 15 ) :
    scenario = "historical"

if ( lyr_perp ) :
    scenario = "perpetual"
    perpetual_year = yr_perp

**Some more initialisation**

In [ ]:
switch_year = False

start_date = isodate.parse_datetime(iso_start_date)
end_datetime = isodate.parse_datetime(iso_end_date)
coupling_interval = isodate.parse_duration(iso_coupling_interval)
data_interval = isodate.parse_duration(iso_data_interval)

**Reading the data**

In [ ]:
def filename_dflt ( dataPath , fileRoot, band ) :
    filename =  dataPath+fileRoot+"_"+band+"_rast.nc"
    return filename

def filename_year ( dataPath , fileRoot, band, year ) :
    if ( year >= 1845 and year <= 2100 ) :
        filename = dataPath+fileRoot+"_"+band+"_"+str(year)+"_rast.nc"
        return filename
    else:
        print ( filename, " not available!" )
        raise SystemExit(1); exit        


In [ ]:
filename = filename_dflt( dataPath, fileRoot, "lw_b16_coa" )

if ( VERBOSE > 0 ) :
    print ( "aero_provider: Reading \n", filename )

lw_b16_coa = xr.open_dataset(filename, decode_times=False)

print ( "lw_b16_coa.dims:", lw_b16_coa.dims["lnwl"] )
print ( "delta_z:        ", lw_b16_coa["delta_z"].values )

filename = filename_dflt( dataPath, fileRoot, "sw_b14_coa" )

if ( VERBOSE > 0 ) :
    print ( "aero_provider: Reading \n", filename )

sw_b14_coa = xr.open_dataset(filename, decode_times=False)

print ( "sw_b14_coa.dims:", sw_b14_coa.dims["lnwl"] )
print ( "delta_z:        ", sw_b14_coa["delta_z"].values )

if ( scenario == 'picontrol' ) :
    filename = filename_year ( dataPath, fileRoot, "sw_b14_fin", 1850 )
if ( scenario == 'perpetual' ) :
    filename = filename_year ( dataPath, fileRoot, "sw_b14_fin", perpetual_year )
if ( scenario == 'historical' ) :
    filename = filename_year ( dataPath, fileRoot, "sw_b14_fin", start_date.year )

if ( VERBOSE > 0 ) :
    print ( "aero_provider: Reading \n", filename )

this_sw_b14_fin = xr.open_dataset(filename, decode_times=False)
other_sw_b14_fin = this_sw_b14_fin

print ( "sw_b14_coa.dims:", sw_b14_coa.dims["lnwl"] )
print ( "delta_z:        ", this_sw_b14_fin["delta_z"].values )

**Time loop**

In [ ]:
this_date = start_date
watch_this = 0
watch_other = 0

while this_date <= end_datetime:

    this_month = this_date.month
    this_year = this_date.year

    if ( this_month < 12) :
        mid_of_this_month = (date(this_year, this_month+1, 1) -
                             date(this_year, this_month, 1)).days * 43200
    else :
        mid_of_this_month = 31 * 43200

    sec_of_this_month = (this_date.day-1) * 86400 + \
                         this_date.hour   * 3600 +  \
                         this_date.minute * 60 +    \
                         this_date.second  
    
    if ( sec_of_this_month <= mid_of_this_month ) :
        switch = -1
    else :
        switch = 1

    other_date = this_date + switch * data_interval
    other_month = other_date.month
    other_year = other_date.year

    if ( scenario == "historical" ) :

        if ( other_year != this_year and not switch_year ) :
            filename = filename_year( dataPath, fileRoot, "sw_b14_fin", other_year )
            other_sw_b14_fin = xr.open_dataset(filename, decode_times=False)
            switch_year = True
            if VERBOSE > 0:
                print ( "aero_provider: Opened dataset for year ", other_year )

        if ( other_year == this_year and switch_year ) :
            switch_year = False
            other_sw_b14_fin = dataset_sw_b14_fin
            if VERBOSE > 0:
                print ( "aero_provider: Complete switch to year ", other_year )

    if ( other_month < 12) :
        mid_of_other_month = (date(this_year, other_month+1, 1) -
                              date(this_year, other_month, 1)).days * 43200
    else :
        mid_of_other_month = 31 * 43200
 
    other_wght = switch * ( sec_of_this_month - mid_of_this_month ) / \
                          ( mid_of_this_month + mid_of_other_month )
    this_wght  = 1 - other_wght

    if ( this_month != watch_this ) :
        if ( watch_this == 0 ) :

            if ( VERBOSE > 1 ) :
                print ( "aero_provider: Reading this month", this_month )

            this_aod_lw_b16_coa  = lw_b16_coa["aod"]["time"][this_month-1].values.reshape((1,-1))
            this_ssa_lw_b16_coa  = lw_b16_coa["ssa"]["time"][this_month-1].values.reshape((1,-1))
            this_asy_lw_b16_coa  = lw_b16_coa["asy"]["time"][this_month-1].values.reshape((1,-1))
            this_aer_lw_b16_coa  = lw_b16_coa["z_aer_coarse_mo"]["time"][this_month-1].values.reshape((1,-1))

            this_aod_sw_b14_coa  = sw_b14_coa["aod"]["time"][this_month-1].values.reshape((1,-1))
            this_ssa_sw_b14_coa  = sw_b14_coa["ssa"]["time"][this_month-1].values.reshape((1,-1))
            this_asy_sw_b14_coa  = sw_b14_coa["asy"]["time"][this_month-1].values.reshape((1,-1))
            this_aer_sw_b14_coa  = sw_b14_coa["z_aer_coarse_mo"]["time"][this_month-1].values.reshape((1,-1))
        
            this_aod_sw_b14_fin  = this_sw_b14_fin["aod"]["time"][this_month-1].values.reshape((1,-1))
            this_ssa_sw_b14_fin  = this_sw_b14_fin["ssa"]["time"][this_month-1].values.reshape((1,-1))
            this_asy_sw_b14_fin  = this_sw_b14_fin["asy"]["time"][this_month-1].values.reshape((1,-1))
            this_aer_sw_b14_fin  = this_sw_b14_fin["z_aer_fine_mo"]["time"][this_month-1].values.reshape((1,-1))

        else :

            if ( VERBOSE > 1 ) :
                print ( "aero_provider: Flipp this month to", this_month )

            this_aod_lw_b16_coa = other_aod_lw_b16_coa
            this_ssa_lw_b16_coa = other_ssa_lw_b16_coa
            this_asy_lw_b16_coa = other_asy_lw_b16_coa
            this_aer_lw_b16_coa = other_aer_lw_b16_coa

            this_aod_sw_b14_coa = other_aod_sw_b14_coa
            this_ssa_sw_b14_coa = other_ssa_sw_b14_coa
            this_asy_sw_b14_coa = other_asy_sw_b14_coa
            this_aer_sw_b14_coa = other_aer_sw_b14_coa

            this_aod_sw_b14_fin = other_aod_sw_b14_fin
            this_ssa_sw_b14_fin = other_ssa_sw_b14_fin
            this_asy_sw_b14_fin = other_asy_sw_b14_fin
            this_aer_sw_b14_fin = other_aer_sw_b14_fin

        watch_this = this_month

    # with an extra copy one could save some more reading from file

    if ( other_month != watch_other ) :

        if ( VERBOSE > 1 ) :
            print ( "aero_provider: Reading other month", other_month )

        other_aod_lw_b16_coa = lw_b16_coa["aod"]["time"][other_month-1].values.reshape((1,-1))
        other_ssa_lw_b16_coa = lw_b16_coa["ssa"]["time"][other_month-1].values.reshape((1,-1))
        other_asy_lw_b16_coa = lw_b16_coa["asy"]["time"][other_month-1].values.reshape((1,-1))
        other_aer_lw_b16_coa = lw_b16_coa["z_aer_coarse_mo"]["time"][other_month-1].values.reshape((1,-1))

        other_aod_sw_b14_coa = sw_b14_coa["aod"]["time"][other_month-1].values.reshape((1,-1))
        other_ssa_sw_b14_coa = sw_b14_coa["ssa"]["time"][other_month-1].values.reshape((1,-1))
        other_asy_sw_b14_coa = sw_b14_coa["asy"]["time"][other_month-1].values.reshape((1,-1))
        other_aer_sw_b14_coa = sw_b14_coa["z_aer_coarse_mo"]["time"][other_month-1].values.reshape((1,-1))

        other_aod_sw_b14_fin = other_sw_b14_fin["aod"]["time"][other_month-1].values.reshape((1,-1))
        other_ssa_sw_b14_fin = other_sw_b14_fin["ssa"]["time"][other_month-1].values.reshape((1,-1))
        other_asy_sw_b14_fin = other_sw_b14_fin["asy"]["time"][other_month-1].values.reshape((1,-1))
        other_aer_sw_b14_fin = other_sw_b14_fin["z_aer_fine_mo"]["time"][other_month-1].values.reshape((1,-1))

        watch_other = other_month

    if ( VERBOSE > 1 ) :
        print ("aero_provider: ", this_date.isoformat() , ' wght ' , 
               "%8.6f" % this_wght, '*' , "%2i" % this_month , '+' ,
               "%8.6f" % other_wght, '*', "%2i" % other_month)

    aod_lw_b16_coa_field = this_wght * this_aod_lw_b16_coa + other_wght * other_aod_lw_b16_coa
    ssa_lw_b16_coa_field = this_wght * this_ssa_lw_b16_coa + other_wght * other_ssa_lw_b16_coa
    asy_lw_b16_coa_field = this_wght * this_asy_lw_b16_coa + other_wght * other_asy_lw_b16_coa
    aer_lw_b16_coa_field = this_wght * this_aer_lw_b16_coa + other_wght * other_aer_lw_b16_coa

    aod_sw_b14_coa_field = this_wght * this_aod_sw_b14_coa + other_wght * other_aod_sw_b14_coa
    ssa_sw_b14_coa_field = this_wght * this_ssa_sw_b14_coa + other_wght * other_ssa_sw_b14_coa
    asy_sw_b14_coa_field = this_wght * this_asy_sw_b14_coa + other_wght * other_asy_sw_b14_coa
    aer_sw_b14_coa_field = this_wght * this_aer_sw_b14_coa + other_wght * other_aer_sw_b14_coa

    aod_sw_b14_fin_field = this_wght * this_aod_sw_b14_fin + other_wght * other_aod_sw_b14_fin
    ssa_sw_b14_fin_field = this_wght * this_ssa_sw_b14_fin + other_wght * other_ssa_sw_b14_fin
    asy_sw_b14_fin_field = this_wght * this_asy_sw_b14_fin + other_wght * other_asy_sw_b14_fin
    aer_sw_b14_fin_field = this_wght * this_aer_sw_b14_fin + other_wght * other_aer_sw_b14_fin

    this_date = this_date + coupling_interval

print ( "The End" )